使用Mido第三方库进行测试，在展开实验前，先规定一些参数

In [1]:
import mido
from random import randint, choice, random
from typing import List, Literal, Union

BPM = 120  # Beats per minute
VELOCITY = 64  # MIDI note velocity
TICK_PER_BEAT = 480  # Tick per beat

# The range of notes to be generated
NOTE_MIN = 48  # C3
NOTE_MAX = 84  # C6

# Note lengths
WHOLE = 4 * TICK_PER_BEAT
HALF = 2 * TICK_PER_BEAT
QUARTER = 1 * TICK_PER_BEAT
EIGHTH = QUARTER // 2
SIXTEENTH = QUARTER // 4

音符的包装——包含起始时间、持续时间、音高、音量的属性，同时提供了音名翻译的接口。

In [2]:
note_name_dict = {
    'C': 0, 'C#': 1, 'Cb': -1,
    'D': 2, 'D#': 3, 'Db': 1,
    'E': 4, 'E#': 5, 'Eb': 3, 
    'F': 5, 'F#': 6, 'Fb': 4,
    'G': 7, 'G#': 8, 'Gb': 6, 
    'A': 9, 'A#': 10, 'Ab': 8,
    'B': 11, 'B#': 12, 'Bb': 10,
}

Key_Major_T = Literal['C', 'Db', 'D', 'Eb', 'E', 'F',
                      'F#', 'E', 'G', 'Ab', 'A', 'Bb', 'B']
Key_Minor_T = Literal['Cm', 'C#m', 'Dm', 'D#m', 'Ebm', 'Em', 
                      'Fm', 'F#m', 'Gm', 'G#m', 'Am', 'Bbm', 'Bm']
Key_T = Union[Key_Major_T, Key_Minor_T]
Pitch_T = int

# offset of the notes in the major mode and minor mode
major_offset = (0, 2, 4, 5, 7, 9, 11)
minor_offset = (0, 2, 3, 5, 7, 8, 10)


class Note:
    def __init__(self, pitch: Pitch_T, length: int, 
                 start_time: int, velocity: int = VELOCITY):
        # Here the "time" is "tick" in mido actually
        self.pitch = pitch
        self.length = length
        self.start_time = start_time
        self.velocity = velocity

    def __str__(self):
        pitch = self.pitch_to_name(self.pitch)
        return f"{pitch}: length={self.length} start={self.start_time} velocity={self.velocity}"

    @staticmethod
    def in_mode(pitch: Pitch_T, key: Key_T):
        """judge if the note is in the given mode."""
        if key.endswith('m'):
            # minor mode
            base = note_name_dict[key[:-1]]
            return (pitch - base) % 12 in minor_offset
        else:
            # major mode
            base = note_name_dict[key]
            return (pitch - base) % 12 in major_offset
            
    @staticmethod
    def name_to_pitch(note_name: str) -> Pitch_T:
        """Convert a note name to a pitch.
        For example, "C4" -> 60. """
        octave = note_name[-1]
        name = note_name[:-1]
        pitch = note_name_dict[name]
        return (int(octave) + 1) * 12 + pitch

    @staticmethod
    def pitch_to_name(pitch: Pitch_T) -> str:
        """Convert a pitch to a note name.
        For example, 60 -> "C4"."""
        octave = pitch // 12 - 1
        note = pitch % 12
        for name, picth in note_name_dict.items():
            if note == picth:
                return name + str(octave)

对音轨的包装
- `self.note`是所有音符构成的列表，直接访问即可
- 提供了与`mido.MidiTrack`相互转化的接口
- 完成了对音轨的移调、倒影、逆行工作

In [3]:
class Track:
    """A wrapper for mido.MidiTrack."""

    def __init__(self, instrument: int = 0, key: Key_T = 'C'):
        self.instrument = instrument
        self.key = key
        self.note: list[Note] = []

    def __str__(self):  # used for debug
        meta_msg = f"Key: {self.key}\nInstrument: {self.instrument}\n"
        note_msg = '\n'.join([str(note) for note in self.note])
        return meta_msg + note_msg

    def print_brief_info(self):
        """Brief information of the track."""
        print(f"Key: {self.key}")
        print(f"Instrument: {self.instrument}")
        print(f"Length: {self.full_length}")
        print(f"Bar: {self.bar}")

    @staticmethod
    def from_track(track: mido.MidiTrack):
        """Generate a track from a midi track.
        Available for chords."""

        ga_track = Track()
        time = 0
        note_dict = {}  # pitch -> (start_time, velocity)
        for msg in track:
            if msg.type == 'program_change':
                ga_track.instrument = msg.program
            elif msg.type == 'key_signature':
                ga_track.key = msg.key
            elif msg.type == 'note_on':
                time += msg.time
                note_dict[msg.note] = (time, msg.velocity)
            elif msg.type == 'note_off':
                time += msg.time
                start_time, velocity = note_dict.pop(msg.note)
                ga_track.note.append(
                    Note(msg.note, time - start_time, start_time, velocity))
        return ga_track

    def to_track(self) -> mido.MidiTrack:
        """Generate a midi track from a track.
        Available for chords."""

        midi_track = mido.MidiTrack()
        midi_track.append(mido.MetaMessage(
            'key_signature', key=self.key, time=0))
        midi_track.append(mido.Message(
            'program_change', program=self.instrument, time=0))

        event_set = set()  # (time, 0/1(note_off/on), note)
        for note in self.note:
            event_set.add((note.start_time, 1, note))
            event_set.add((note.start_time + note.length, 0, note))
        # Sort all the events by time. If the time is same, note_off is before note_on.
        sorted_time = sorted(event_set, key=lambda x: (x[0], x[1]))

        last_time = 0
        for time, event_num, note in sorted_time:
            event = 'note_on' if event_num else 'note_off'
            msg = mido.Message(
                event, note=note.pitch, velocity=note.velocity, time=time-last_time)
            midi_track.append(msg)
            last_time = time
        return midi_track

    def generate_random_track(self, bar: int):
        """Generate a random track with the given bar length."""
        for i in range(bar):
            length = WHOLE
            while length > 0:
                l = choice([WHOLE, HALF, QUARTER, EIGHTH, SIXTEENTH])
                if l <= length:
                    pitch = randint(NOTE_MIN, NOTE_MAX)
                    while not Note.in_mode(pitch, self.key):
                        pitch = randint(NOTE_MIN, NOTE_MAX)
                    note = Note(pitch, l, i * WHOLE + WHOLE - length)
                    length -= l
                    self.note.append(note)

    @property
    def full_length(self):
        """The length of the track"""
        return max(note.start_time + note.length for note in self.note)

    @property
    def bar(self):
        """The number of bars"""
        return self.full_length // WHOLE + 1

    def transpose(self, interval):
        """Transpose the track by the given interval"""
        for note in self.note:
            note.pitch += interval

    def inverse(self, center):
        """Inverse the track by the given center"""
        for note in self.note:
            note.pitch = 2 * center - note.pitch

    def retrograde(self):
        """Retrograde the track"""
        for note in self.note:
            note.start_time = self.full_length - note.start_time - note.length
        self.note.reverse()

测试函数
- `get_midi` 生成一个给定调式的空白midi，同时建立好调式、曲速、调号的元信息
- `generate_random_midi` 生成一个随机midi，调式指定为#g小调
- `read_midi` 读取一个现有的midi——久石让的《Summer》间奏片段。其中包含和弦、音符强弱变化以用于测试

In [4]:
def get_midi(key: Key_T = None):
    s = mido.MidiFile()
    meta_track = mido.MidiTrack()
    meta_track.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(BPM)))
    meta_track.append(mido.MetaMessage(
        'time_signature', numerator=4, denominator=4))
    if key is not None:
        meta_track.append(mido.MetaMessage(
            'key_signature', key=key, time=0))
    s.tracks.append(meta_track)
    return s

In [5]:
from copy import deepcopy


def generate_random_midi():
    s = get_midi()
    track = Track(0, 'G#m')
    track.generate_random_track(4)  # 4 bars
    s.tracks.append(track.to_track())
    retrograde_track = deepcopy(track)  # MUST use deepcopy
    retrograde_track.retrograde()
    s.tracks.append(retrograde_track.to_track())
    s.save('random.mid')

    # Expected result:
    # random.mid: A random piece with 4 bars in G sharp minor which has 2 tracks,
    # one is the original track, the other is the retrograde track.


def read_midi():
    s = mido.MidiFile('test.mid')
    right_hand = Track.from_track(s.tracks[0])
    right_hand.print_brief_info()
    print('----------------')
    left_hand = Track.from_track(s.tracks[1])
    left_hand.print_brief_info()
    # For more information, use print(track) to see the detailed notes.
    print(left_hand)

    # Expected output:
    # Key: D
    # Instrument: 0
    # Length: 15353
    # Bar: 8
    # ----------------
    # Key: D
    # Instrument: 0
    # Length: 15347
    # Bar: 8

generate_random_midi()
read_midi()

Key: D
Instrument: 0
Length: 15353
Bar: 8
----------------
Key: D
Instrument: 0
Length: 15347
Bar: 8
Key: D
Instrument: 0
D3: length=455 start=0 velocity=49
F#3: length=455 start=0 velocity=49
A3: length=455 start=0 velocity=49
D4: length=455 start=0 velocity=49
D3: length=455 start=480 velocity=49
F#3: length=455 start=480 velocity=49
A3: length=455 start=480 velocity=49
D4: length=455 start=480 velocity=49
E3: length=455 start=960 velocity=49
G3: length=455 start=960 velocity=49
A3: length=455 start=960 velocity=49
D4: length=455 start=960 velocity=49
E3: length=455 start=1440 velocity=49
G3: length=455 start=1440 velocity=49
A3: length=455 start=1440 velocity=49
D4: length=455 start=1440 velocity=49
F#3: length=455 start=1920 velocity=49
A3: length=455 start=1920 velocity=49
D4: length=455 start=1920 velocity=49
F#3: length=455 start=2400 velocity=49
A3: length=455 start=2400 velocity=49
D4: length=455 start=2400 velocity=49
G3: length=341 start=2880 velocity=49
A3: length=341 start

接下来做遗传算法的准备工作

In [6]:
class TrackPopulation:

    ChromosomeType = Track

    def __init__(self, population: List[ChromosomeType], mutation_rate: float = 0.1):
        self.population = population
        self.mutation_rate = mutation_rate
        self.adaptability = [0] * len(population)
        self._update_adaptability()
        self.best_index, self.second_index = 0,0

    def _update_adaptability(self):
        pass

    def select(self):
        for i in range(len(self.adaptability)):
            if self.adaptability[i] > self.adaptability[self.best_index]:
                self.best_index = i
                self.second_index = self.best_index
            elif self.adaptability[i] > self.adaptability[self.second_index]:
                self.second_index = i

    def crossover(self):
        pass

    def mutate(self):
        for i in range(len(self.population)):
            if random() > self.mutation_rate:
                continue
            # TODO: mutation
            pass
        self._update_adaptability()

    def show_info(self):
        pass

    def run(self, generation):
        for i in range(generation):
            print(f"<epoch {i} begins>")
            self.crossover()
            self.mutate()
            self.show_info()
            print(f"<epoch {i} ends>")
            
            # TODO: target parameter and return
            target = 0 
            if self.adaptability[self.best_index] >= target:
                print(f"[!] Target reached at epoch {i}")
                return self.population[self.best_index]

遗传算法的运行测试

In [7]:
from time import time

KEY = 'G'
t0 = time()

population = [Track(0, KEY).generate_random_track(4) for _ in range(10)]
ga = TrackPopulation(population, 0.2)
result = ga.run(1000)

s = get_midi(KEY)
s.tracks.append(result.to_track())
s.save('result.mid')

print(f"Time cost: {time() - t0}s")

<epoch 0 begins>
<epoch 0 ends>
[!] Target reached at epoch 0


AttributeError: 'NoneType' object has no attribute 'to_track'